In [1]:
import sys
import logging
import os
import time
import boto3
import json

In [3]:
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 

In [8]:
kb_name = 'demo-knowledge-base-csv'
kbs = bedrock_agent_client.list_knowledge_bases(maxResults=100)
kb_id = next((kb['knowledgeBaseId'] for kb in kbs['knowledgeBaseSummaries'] if kb['name'] == kb_name), None)
kb_id

'FJG5JJM6J1'

In [25]:
query = "What are key drink categories?"
fm = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{fm}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                }
            }
        }
    }
)
print(f"Response status: {response['ResponseMetadata']['HTTPStatusCode']}")

Response status: 200


In [27]:
print(f"""
Here is the query:
{query}

Here is the NL output:
{response['output']['text']}

Here is the SQL to retrieve data from CSV files:
{response['citations'][0]['retrievedReferences'][0]['location']['sqlLocation']['query']}
""")


Here is the query:
What are key drink categories?

Here is the NL output:
The key drink categories based on the query output are Coffee, Espresso, Tea, and Herbal Tea. The caffeination levels for these categories are either Regular or Decaf.

Here is the SQL to retrieve data from CSV files:
SELECT DISTINCT "category", "caffeination" FROM awsdatacatalog."demo-kb".demo_csv_639649899411;

